In [ ]:
# versions: pandas==2.0.3, nibabel==5.2.1, numpy==1.24.4, scipy==1.8.0

In [ ]:
import pandas as pd
import os
import nibabel as nib
import numpy as np
from scipy.ndimage import gaussian_filter


In [ ]:
DATASET_TRAIN="./data/CN+MCI_BL+VOL_train.csv"
DATASET_TEST="./data/CN+MCI_BL+VOL_test.csv"
FASTSURFER_OUTPUT_DIR="./data/ADNI/output_fastsurfer/"
VBM_RESULTS_DIR="./results/VBM/ADNI/"

In [ ]:
data_aseg=pd.read_table(FASTSURFER_OUTPUT_DIR+"aseg_stats.txt")


data_aseg=data_aseg.rename({"Measure:volume":"filename"},axis=1)
data_aseg["IMAGEUID"]=data_aseg.filename.str.split("_I").str[-1].str[:-1]
data_aseg=data_aseg.astype({"IMAGEUID": "str"})
data_aseg=data_aseg.filter(["filename","IMAGEUID"])
paths=FASTSURFER_OUTPUT_DIR+data_aseg.filename+"/mri/mri/mwp1norm.nii"
data_aseg["filename_mwp1"]=paths

In [ ]:
train=pd.read_csv(DATASET_TRAIN)
test=pd.read_csv(DATASET_TEST)

In [ ]:
trainvalidDF=pd.concat([train,test],ignore_index=True)
trainvalidDF=trainvalidDF.astype({"IMAGEUID": "str"})


In [ ]:
trainvalidDF=pd.merge(trainvalidDF,data_aseg,on="IMAGEUID",how="left")

In [ ]:
def fwhm2sigma(fwhm):
    return fwhm / np.sqrt(8 * np.log(2))
sigma = fwhm2sigma(8/1.5)

In [ ]:
i=0
for index,dft in trainvalidDF.iterrows():
    age=dft["AGE"]
    gender=dft["PTGENDER_Male"]
    time=dft["Time"]
    status=dft["Status"]
    ptid=dft["PTID"]
    imageuid=dft["IMAGEUID"]
    if not os.path.isfile("./Results_VBM/FastSurfer_Results_TimeToEvent_all_Untersuchung_TrainValidTest_pixel_1749352.csv"):
        template_img = nib.load(dft["filename_mwp1"])
        template_data = np.asanyarray(template_img.dataobj)
        smoothed_volume = gaussian_filter(template_data, sigma)

        fieldstrength=dft["MagStrength_3.0"]
        eTIV=dft["EstimatedTotalIntraCranialVol"]
        for index_pixel in range(0,smoothed_volume.flatten().shape[0]):
            with open("./Results_VBM/FastSurfer_Results_TimeToEvent_all_Untersuchung_TrainValidTest_pixel_"+str(index_pixel)+".csv", "ab") as f:
                np.savetxt(f,[[smoothed_volume.flatten()[index_pixel],age,gender,time,status,fieldstrength,eTIV,ptid,imageuid]], delimiter=",", fmt="%s")
                f.write(b"\n")
    else:
        df=pd.read_csv("./Results_VBM/FastSurfer_Results_TimeToEvent_all_Untersuchung_TrainValidTest_pixel_1749352.csv",header=None)
        df.columns=["pixel","age","gender","time","status","fieldstrength","eTIV","ptid","imageuid"]
        if not int(imageuid) in df.imageuid.tolist():
            template_img = nib.load(dft["filename_mwp1"])
            template_data = np.asanyarray(template_img.dataobj)
            smoothed_volume = gaussian_filter(template_data, sigma)
            fieldstrength=dft["MagStrength_3.0"]
            eTIV=dft["EstimatedTotalIntraCranialVol"]
            for index_pixel in range(0,smoothed_volume.flatten().shape[0]):
                with open(VBM_RESULTS_DIR+"FastSurfer_Results_TimeToEvent_all_Untersuchung_TrainValidTest_pixel_"+str(index_pixel)+".csv", "ab") as f:
                    np.savetxt(f,[[smoothed_volume.flatten()[index_pixel],age,gender,time,status,fieldstrength,eTIV,ptid,imageuid]], delimiter=",", fmt="%s")
                    f.write(b"\n")